In [ ]:
import ROOT

# A simple helper function to fill a test tree: this makes the example stand-alone.
rsops = ROOT.ROOT.RDF.RSnapshotOptions(
         "RECREATE",       # mode
         ROOT.ROOT.kZLIB,  # compression algorithm
         1,                # compression level
         200,              # autoflush, number of events per cluster
         99,               # split level of output tree
         0                 # lazy
        )

def fill_tree(treeName, fileName):
    rdf = ROOT.ROOT.RDataFrame(500000)
    rdf.Define("b1", "(double) tdfentry_")\
       .Define("b2", "(int) tdfentry_ * tdfentry_")\
       .Snapshot(treeName, fileName, "", rsops)

In [ ]:
# We prepare an input tree to run on
fileName = "exercise2.root"
treeName = "myTree"
fill_tree(treeName, fileName)

In [ ]:
# PyRDF, python library to run RDataFrame-based analysis on Spark
import PyRDF

# Setup a Spark session
from pyspark import SparkContext

context = SparkContext.getOrCreate(conf=swan_spark_conf)
context.stop()

# Define Spark as the backend for PyRDF
PyRDF.use("spark")

In [ ]:
# We read the tree from the file and create a RDataFrame, a class that
# allows us to interact with the data contained in the tree.
RDF = PyRDF.RDataFrame
d = RDF(treeName, fileName)


In [ ]:
# Create a histogram with values from column 'b2'
hist1 = d.Filter('b1 > 20000 && b1 < 100000')\
         .Histo1D(("MyHisto", "", 100, 10000, 150000), "b1")

In [ ]:
# Launch Spark computation
c1 = ROOT.TCanvas()
hist1.Draw()
c1.Draw()